# Examen Final - Teoría de Sistemas de Control - UTEC - 2020-1

**Alumno**: Bregy Malpartida

**Stack**: Python3.8, Numpy, Matplotlib, Control, Control2020, Scipy, Jupyter, IPython

---

In [1]:
import numpy as np
import control as ct
import control2020 as ct20
from matplotlib import pyplot as plt
%matplotlib inline

## Pregunta 1
Se tiene el siguiente sistema en lazo abierto
$$G(s)=\frac{0.3(s+2)}{s(s+45)(s+6.2)(s+0.7)}$$

### 1.a
Diseñe un compensador en la frecuencia de modo a obtener un Kv de 40 y un %OS menor al 10%. Justifique que compensador utilizará. Muestre la gráfica del error a la rampa y el diagrama de bode indicando los márgenes. Use el tiempo de simulación que crea conveniente.

**Solución**

In [3]:
s = ct.TransferFunction([1, 0], [1])
g = 0.3*(s+2)/s/(s+45)/(s+6.2)/(s+0.7)
display(g)


            0.3 s + 0.6
------------------------------------
s^4 + 51.9 s^3 + 314.8 s^2 + 195.3 s